In [204]:
import swat
import pandas as pd
import os
%run "C:\\Users\\frajoz\\Programs\\Scripts\\connection.py"
s = connection_CAS("VIYAWAVES","10.255.154.254")
pd.set_option('display.max_colwidth',1000)
model='N'

In [205]:


if model=='N':
    path="C:\\Users\\frajoz\\OneDrive - SAS\\Domain\\Security Intelligence\\Demo\\DATA INTERLIKE\\"
    docid='docid'
    for doc in os.listdir(path):
        if ".csv" in doc:            
            if 'CONCEPT' in doc.upper():
                s.upload_frame(pd.read_csv(path+doc,sep=","),casout={'name':'OUTCONCEPT','caslib':'CASUSER','replace':True})
            elif 'FACT' in doc.upper():
                s.upload_frame(pd.read_csv(path+doc,sep=","),casout={'name':'OUTFACT','caslib':'CASUSER','replace':True})



elif model=='Y':
    table='FACTIVA_DATA'
    var_text='LP'
    caslib='Public'
    docid='Unique_ID'
    s.table.loadTable(path=table+".sashdat",caslib=caslib,casout={'name':table,'replace':True})
    liti_binary_caslib = "Analytics_Project_39fd00f8-e182-4283-81e1-9c113a69ebda"
    liti_binary_table_name = "8a7f0b047bfeff74017c55f795d80110_CONCEPT_BINARY"
    s.builtins.loadActionSet(actionSet="textRuleDevelop")
    s.builtins.loadActionSet(actionSet="textRuleScore")
    s.textRuleScore.applyConcept(casOut={"name":"OUTCONCEPT", "replace":True},     # 4
                                 docId=docid,
                                 factOut={"name":"OUTFACT", "replace":True},
                                 model={"name":liti_binary_table_name,"caslib":liti_binary_caslib},
                                 ruleMatchOut={"name":"out_rule_match", 
                                               "replace":True
                                              },
                                 matchType='ALL',
                                 table={"name":table},
                                 language='FRENCH',
                                 text=var_text
                                )
print(docid)

NOTE: Cloud Analytic Services made the uploaded file available as table OUTCONCEPT in caslib CASUSER(frajoz).
NOTE: The table OUTCONCEPT has been created in caslib CASUSER(frajoz) from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table OUTFACT in caslib CASUSER(frajoz).
NOTE: The table OUTFACT has been created in caslib CASUSER(frajoz) from binary data uploaded to Cloud Analytic Services.
docid


### CONCEPTS DEDUP

In [206]:

s.datastep.runcode('data CASUSER.CONCEPTS_CALCUL;set CASUSER.OUTCONCEPT ;by '+docid+' _concept_ _start_;prevstart=lag(_start_);prevend=lag(_end_);L_MATCH=length(_match_text_);retain result_id 1;if (_start_>prevend+3 or _start_<prevstart-3) then do;result_id+1;end;run;')
s.builtins.loadActionSet("fedSql")
s.fedsql.execDirect(query='''create table CONCEPTS_MAX{options replace=true} as 
select distinct '''+docid+''',_path_,_concept_,result_id,max(L_MATCH) as L_MAX
from CONCEPTS_CALCUL group by '''+docid+''',_path_,_concept_,result_id;''')

s.fedsql.execDirect(query='''create table CONCEPTS_DEDUP{options replace=true} as 
select t1.*,t2._start_,t2._end_,t2._match_text_,t2._canonical_form_
from CONCEPTS_MAX t1 left join CONCEPTS_CALCUL t2 on t1.'''+docid+'''=t2.'''+docid+''' and t1.result_id=t2.result_id 
and t1.L_MAX=t2.L_MATCH; ''')


NOTE: Missing values were generated as a result of performing an operation on missing values.
      Each place is given by: (Number of times) at (Line):(Column).
      1 at 0:189
NOTE: Duplicate messages output by DATA step:
NOTE: Missing values were generated as a result of performing an operation on missing values.  (occurred 6 times)
      Each place is given by: (Number of times) at (Line):(Column).  (occurred 6 times)
      1 at 0:189  (occurred 6 times)
NOTE: Added action set 'fedSql'.
NOTE: Table CONCEPTS_MAX was created in caslib CASUSER(frajoz) with 171 rows returned.
NOTE: Table CONCEPTS_DEDUP was created in caslib CASUSER(frajoz) with 175 rows returned.


+ Elapsed: 0.186s, user: 0.202s, sys: 0.143s, mem: 133mb

### FACTS DEDUP

In [207]:
s.fedsql.execDirect(query='''create table OUTFACT_COUNT{options replace=true} as 
select '''+docid+''',_result_id_,_fact_,count(distinct _match_text_)-1 as cpt_arg
from OUTFACT group by '''+docid+''',_result_id_,_fact_ ''')
s.fedsql.execDirect(query='''create table FACTS_ARGUMENTS{options replace=true} as 
select t1.*
from OUTFACT t1 inner join CONCEPTS_DEDUP t2 on t1.'''+docid+'''=t2.'''+docid+''' and t1._match_text_=t2._match_text_; ''')

s.fedsql.execDirect(query='''create table fact_count{options replace=true} as 
select distinct '''+docid+''',_fact_ ,_result_id_,count(distinct _match_text_) as CT_R
from FACTS_ARGUMENTS group by '''+docid+''',_fact_,_result_id_''')

s.fedsql.execDirect(query='''create table factjoin{options replace=true} as 
select distinct t1.* from fact_count t1 right join OUTFACT_COUNT t2 on t1.'''+docid+'''=t2.'''+docid+''' and 
t1._result_id_=t2._result_id_ and t1.CT_R=t2.cpt_arg''')

s.fedsql.execDirect(query='''create table factjoin_full{options replace=true} as 
select distinct t1.'''+docid+''',t1._fact_,t1._result_id_,t2._fact_argument_,t2._start_,t2._end_,t2._match_text_,t2._path_ 
from FACTJOIN t1 left join OUTFACT t2 on t1.'''+docid+'''=t2.'''+docid+''' and t1._result_id_=t2._result_id_ where t1._fact_ is not null''')

s.datastep.runcode('data CASUSER.FACTS_CALCUL;set CASUSER.factjoin_full(where=(_fact_argument_="")) ;by '+docid+' _fact_ _start_;prevstart=lag(_start_);prevend=lag(_end_);L_MATCH=length(_match_text_);retain result_id 1;if (_start_>prevend+3 or _start_<prevstart-3) then do;result_id+1;end;run;')


s.fedsql.execDirect(query='''create table fact_nearest1{options replace=true} as 
select distinct '''+docid+''',_fact_,result_id,min(L_match) as min_L from FACTS_CALCUL group by '''+docid+''',_fact_,result_id''')


s.fedsql.execDirect(query='''create table fact_nearest{options replace=true} as 
select distinct t1.* from FACTS_CALCUL t1 right join FACT_NEAREST1 t2 on t1.'''+docid+'''=t2.'''+docid+''' and t1._fact_=t2._fact_ and 
t1.result_id=t2.result_id and t1.L_match=t2.min_L''')


s.fedsql.execDirect(query='''create table FACTS_NEAREST_FULLMATCH{options replace=true} as 
select distinct t1.'''+docid+''',t1._fact_,t1._result_id_,t1._fact_argument_,t1._start_,t1._end_,t1._match_text_,t1._path_ from OUTFACT t1 
right join fact_nearest t2 
on t1.'''+docid+'''=t2.'''+docid+''' and t1._fact_=t2._fact_ and t1._result_id_=t2._result_id_''')

NOTE: Table OUTFACT_COUNT was created in caslib CASUSER(frajoz) with 464 rows returned.
NOTE: Table FACTS_ARGUMENTS was created in caslib CASUSER(frajoz) with 1016 rows returned.
NOTE: Table FACT_COUNT was created in caslib CASUSER(frajoz) with 420 rows returned.
NOTE: Table FACTJOIN was created in caslib CASUSER(frajoz) with 137 rows returned.
NOTE: Table FACTJOIN_FULL was created in caslib CASUSER(frajoz) with 452 rows returned.
NOTE: Missing values were generated as a result of performing an operation on missing values.
      Each place is given by: (Number of times) at (Line):(Column).
      1 at 0:214
NOTE: Duplicate messages output by DATA step:
NOTE: Missing values were generated as a result of performing an operation on missing values.  (occurred 6 times)
      Each place is given by: (Number of times) at (Line):(Column).  (occurred 6 times)
      1 at 0:214  (occurred 6 times)
NOTE: Table FACT_NEAREST1 was created in caslib CASUSER(frajoz) with 50 rows returned.
NOTE: Table FA

+ Elapsed: 0.227s, user: 0.313s, sys: 0.244s, mem: 137mb

In [208]:

df=s.CASTable('factjoin_full').to_frame()
df.sort_values(by=[docid,'_fact_','_result_id_','_fact_argument_']).tail(50)

,docid,_fact_,_result_id_,_fact_argument_,_start_,_end_,_match_text_,_path_
362,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,38.0,,452.0,1015.0,"Laurent Dupont, né le 07 juin 1978 à Rennes, domicilié au 11 boulevard Pasteur à Fresnes, n° de passeport : 18JO64710, fils de Jeannette Dupont et Robert Dupont.· Nicole Dupont, née le 8 mai 1980 à Tourcoing, réside au 11 boulevard Pasteur à Fresnes, n° de passeport 87AJ86754, femme de Laurent Dupont.Nous avons entrepris une fouille du véhicule et saisi 8.4 kilogrammes de cocaïne. Une perquisition nous a permis de constater des liens avec un dénommé Ludovic Michel, alias Michou, qu’ils devaient rencontrer à la frontière entre la France",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
138,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,38.0,person,452.0,1015.0,Laurent Dupont,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
112,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,38.0,placenaiss,452.0,1015.0,France,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
398,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,45.0,,947.0,1030.0,"Michou, qu’ils devaient rencontrer à la frontière entre la France et la Belgique",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
162,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,45.0,person,947.0,1030.0,Michou,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
220,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,45.0,placenaiss,947.0,1030.0,Belgique,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
197,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,46.0,,947.0,1015.0,"Michou, qu’ils devaient rencontrer à la frontière entre la France",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
198,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,46.0,person,947.0,1015.0,Michou,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
22,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,46.0,placenaiss,947.0,1015.0,France,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE
399,124f3404-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,49.0,,925.0,1030.0,"Ludovic Michel, alias Michou, qu’ils devaient rencontrer à la frontière entre la France et la Belgique",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE


In [127]:

df=s.CASTable('FACTS_CALCUL').to_frame()
df.sort_values(by=[docid,'_fact_','result_id']).head(10)

,docid,_result_id_,_fact_,_fact_argument_,_start_,_end_,_match_text_,_path_,prevstart,prevend,L_MATCH,result_id
416,11a9c302-3cc8-11ec-ab83-121ecb55427b,26.0,C_INDIV_IDENTITE,,428.0,561.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341",C_INDIV/C_INDIV_IDENTITE,NaN,NaN,130.0,2.0
417,11a9c302-3cc8-11ec-ab83-121ecb55427b,25.0,C_INDIV_IDENTITE,,428.0,561.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341",C_INDIV/C_INDIV_IDENTITE,428.0,561.0,130.0,2.0
418,11a9c302-3cc8-11ec-ab83-121ecb55427b,27.0,C_INDIV_IDENTITE,,440.0,561.0,"Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341",C_INDIV/C_INDIV_IDENTITE,428.0,561.0,118.0,2.0
419,11a9c302-3cc8-11ec-ab83-121ecb55427b,8.0,C_INDIV_NAISSANCE,,428.0,477.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing",C_INDIV/C_INDIV_NAISSANCE,440.0,561.0,48.0,3.0
420,11a9c302-3cc8-11ec-ab83-121ecb55427b,9.0,C_INDIV_NAISSANCE,,428.0,477.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing",C_INDIV/C_INDIV_NAISSANCE,428.0,477.0,48.0,3.0
421,11a9c302-3cc8-11ec-ab83-121ecb55427b,7.0,C_INDIV_NAISSANCE,,440.0,477.0,"Boivin, né le 12/12/1978 à Tourcoing",C_INDIV/C_INDIV_NAISSANCE,428.0,477.0,36.0,3.0
422,11a9c302-3cc8-11ec-ab83-121ecb55427b,30.0,C_INDIV_NAISSANCE_DATE,,428.0,464.0,"Jean-Michel Boivin, né le 12/12/1978",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_DATE,440.0,477.0,36.0,4.0
423,11a9c302-3cc8-11ec-ab83-121ecb55427b,29.0,C_INDIV_NAISSANCE_DATE,,428.0,464.0,"Jean-Michel Boivin, né le 12/12/1978",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_DATE,428.0,464.0,36.0,4.0
424,11a9c302-3cc8-11ec-ab83-121ecb55427b,28.0,C_INDIV_NAISSANCE_DATE,,440.0,464.0,"Boivin, né le 12/12/1978",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_DATE,428.0,464.0,24.0,4.0
425,11a9c302-3cc8-11ec-ab83-121ecb55427b,19.0,C_INDIV_NAISSANCE_PLACE,,428.0,485.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing (France",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_PLACE,440.0,464.0,56.0,5.0


In [131]:

df=s.CASTable('FACTS_MAX').to_frame()
df.sort_values(by=[docid,'_fact_','result_id']).head(10)

,docid,_fact_,result_id,L_MAX
41,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_IDENTITE,2.0,130.0
49,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE,3.0,48.0
33,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_DATE,4.0,36.0
45,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,5.0,211.0
50,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_PLACE,6.0,144.0
28,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_RELATIONS,7.0,101.0
34,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_MAILS,8.0,81.0
30,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_MOUVEMENT,9.0,211.0
31,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_MVT_DATE,10.0,376.0
42,11a9c302-3cc8-11ec-ab83-121ecb55427b,DCO_DRUGS_SAISIE,10.0,22.0


In [85]:
df1=s.CASTable('FAC').to_frame()
df1.sort_values(by=[docid,'_concept_','result_id']).head(10)

,docid,_path_,_concept_,result_id,_start_,_end_,_match_text_,_canonical_form_
9,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_SPEAKERS/C_MAILS/C_MAIL_DATE,C_MAIL_DATE,2.0,106.0,115.0,2014/08/07,
25,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_SPEAKERS/C_MAILS/C_MAIL_FROM,C_MAIL_FROM,3.0,71.0,96.0,jeanmarie.theure@police.fr,
152,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_SPEAKERS/C_MAILS/C_MAIL_TO,C_MAIL_TO,4.0,133.0,153.0,secretariat@police.fr,
122,11a9c302-3cc8-11ec-ab83-121ecb55427b,DCO_FACTS/DCO_PERSONNE_IMPLIQUEE,DCO_PERSONNE_IMPLIQUEE,5.0,428.0,445.0,Jean-Michel Boivin,
83,11a9c302-3cc8-11ec-ab83-121ecb55427b,DCO_FACTS/DCO_REF,DCO_REF,6.0,169.0,179.0,IBHEMAIERE1,
123,11a9c302-3cc8-11ec-ab83-121ecb55427b,HELPERS/H_CITOYEN,H_CITOYEN,7.0,337.0,344.0,individu,
84,11a9c302-3cc8-11ec-ab83-121ecb55427b,HELPERS/H_CITOYEN,H_CITOYEN,8.0,689.0,697.0,individus,
112,11a9c302-3cc8-11ec-ab83-121ecb55427b,HELPERS/H_MOMENT,H_MOMENT,9.0,106.0,115.0,2014/08/07,
76,11a9c302-3cc8-11ec-ab83-121ecb55427b,HELPERS/H_MOMENT,H_MOMENT,10.0,253.0,262.0,30/07/2018,
26,11a9c302-3cc8-11ec-ab83-121ecb55427b,HELPERS/H_MOMENT,H_MOMENT,11.0,455.0,464.0,12/12/1978,


In [69]:

dffact=s.CASTable('FACT').to_frame()
dffact.sort_values(by=[docid,'_fact_','_result_id_','_fact_argument_'])[dffact._fact_=='C_MVT_DATE'].head(40)

C:\Users\frajoz\Anaconda3\lib\site-packages\swat\dataframe.py:497: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(SASDataFrame, self).__getitem__(*args, **kwargs)


,docid,_result_id_,_fact_,_fact_argument_,_start_,_end_,_match_text_,_path_
2671,e3754028-3c96-11ec-9c88-121ecb55427b,1.0,C_MVT_DATE,,253.0,645.0,"30/07/2018 à 18:07, dans la ville de Tourcoing (France), nous avons interpellé un individu en possession de 180 grammes de cocaïne. Nous avons identifié le dénommé Jean-Michel Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341, réside au 7 rue Michel Raillard 59200 Tourcoing. Il déclarait se rendre à Lille",C_MOUVEMENT/C_MVT_DATE
2674,e3754028-3c96-11ec-9c88-121ecb55427b,1.0,C_MVT_DATE,date,253.0,645.0,30/07/2018,C_MOUVEMENT/C_MVT_DATE
2673,e3754028-3c96-11ec-9c88-121ecb55427b,1.0,C_MVT_DATE,move,253.0,645.0,se rendre,C_MOUVEMENT/C_MVT_DATE
2675,e3754028-3c96-11ec-9c88-121ecb55427b,1.0,C_MVT_DATE,person,253.0,645.0,Jean-Michel,C_MOUVEMENT/C_MVT_DATE
2672,e3754028-3c96-11ec-9c88-121ecb55427b,1.0,C_MVT_DATE,to,253.0,645.0,Lille,C_MOUVEMENT/C_MVT_DATE
2676,e3754028-3c96-11ec-9c88-121ecb55427b,2.0,C_MVT_DATE,,253.0,645.0,"30/07/2018 à 18:07, dans la ville de Tourcoing (France), nous avons interpellé un individu en possession de 180 grammes de cocaïne. Nous avons identifié le dénommé Jean-Michel Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341, réside au 7 rue Michel Raillard 59200 Tourcoing. Il déclarait se rendre à Lille",C_MOUVEMENT/C_MVT_DATE
2679,e3754028-3c96-11ec-9c88-121ecb55427b,2.0,C_MVT_DATE,date,253.0,645.0,30/07/2018 à 18:07,C_MOUVEMENT/C_MVT_DATE
2678,e3754028-3c96-11ec-9c88-121ecb55427b,2.0,C_MVT_DATE,move,253.0,645.0,se rendre,C_MOUVEMENT/C_MVT_DATE
2680,e3754028-3c96-11ec-9c88-121ecb55427b,2.0,C_MVT_DATE,person,253.0,645.0,Jean-Michel,C_MOUVEMENT/C_MVT_DATE
2677,e3754028-3c96-11ec-9c88-121ecb55427b,2.0,C_MVT_DATE,to,253.0,645.0,Lille,C_MOUVEMENT/C_MVT_DATE


In [107]:
dffact1=s.CASTable('FACTS_DEDUP').to_frame()
dffact1.sort_values(by=[docid,'_fact_','result_id','_fact_argument_']).head(40)

,docid,_fact_,result_id,_fact_argument_,_start_,_end_,_match_text_,_path_
83,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_IDENTITE,2.0,,428.0,561.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341",C_INDIV/C_INDIV_IDENTITE
179,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_IDENTITE,2.0,,440.0,561.0,"Boivin, né le 12/12/1978 à Tourcoing (France), fils de Raymond Boivin et Madeleine Boivin, n° de passeport : 18EM21341",C_INDIV/C_INDIV_IDENTITE
164,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_IDENTITE,2.0,identite,440.0,561.0,18EM21341,C_INDIV/C_INDIV_IDENTITE
140,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE,3.0,,440.0,477.0,"Boivin, né le 12/12/1978 à Tourcoing",C_INDIV/C_INDIV_NAISSANCE
200,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE,3.0,,428.0,477.0,"Jean-Michel Boivin, né le 12/12/1978 à Tourcoing",C_INDIV/C_INDIV_NAISSANCE
249,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE,3.0,datenaiss,440.0,477.0,12/12/1978,C_INDIV/C_INDIV_NAISSANCE
31,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE,3.0,placenaiss,440.0,477.0,Tourcoing,C_INDIV/C_INDIV_NAISSANCE
17,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_DATE,4.0,,440.0,464.0,"Boivin, né le 12/12/1978",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_DATE
68,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_DATE,4.0,,428.0,464.0,"Jean-Michel Boivin, né le 12/12/1978",C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_DATE
208,11a9c302-3cc8-11ec-ab83-121ecb55427b,C_INDIV_NAISSANCE_DATE,4.0,date,440.0,464.0,12/12/1978,C_INDIV/C_INDIV_NAISSANCE/C_INDIV_NAISSANCE_DATE


In [92]:
dfact=s.CASTable('FACT_COUNT').to_frame()
dfact[(dfact._fact_=='C_ACTION')].sort_values(by=['_result_id_'])

,docid,_fact_,_result_id_,CT_R
33,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,22.0,1
384,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,23.0,2
398,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,24.0,2
411,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,25.0,2
323,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,26.0,3
337,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,27.0,3
99,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,28.0,1
34,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,29.0,2
263,11b3ab1a-3cc8-11ec-ab83-121ecb55427b,C_ACTION,30.0,2
297,124f3404-3cc8-11ec-ab83-121ecb55427b,C_ACTION,64.0,3


In [43]:
dfact=s.CASTable('FA').to_frame()
dfact[(dfact._fact_=='C_ACTION')&(dfact.docid=="5b5fa416-3c8d-11ec-bc98-121ecb55427b")&(dfact._result_id_==119.0)].sort_values(by=['_fact_'])

,docid,_result_id_,_fact_,_fact_argument_,_start_,_end_,_match_text_,_path_
1372,5b5fa416-3c8d-11ec-bc98-121ecb55427b,119.0,C_ACTION,,623.0,952.0,"Nicole Dupont, née le 8 mai 1980 à Tourcoing, réside au 11 boulevard Pasteur à Fresnes, n° de passeport 87AJ86754, femme de Laurent Dupont.Nous avons entrepris une fouille du véhicule et saisi 8.4 kilogrammes de cocaïne. Une perquisition nous a permis de constater des liens avec un dénommé Ludovic Michel, alias Michou",C_ACTION
1373,5b5fa416-3c8d-11ec-bc98-121ecb55427b,119.0,C_ACTION,person,623.0,952.0,Michou,C_ACTION
1374,5b5fa416-3c8d-11ec-bc98-121ecb55427b,119.0,C_ACTION,action,623.0,952.0,liens avec,C_ACTION
1375,5b5fa416-3c8d-11ec-bc98-121ecb55427b,119.0,C_ACTION,interpelle,623.0,952.0,Nicole,C_ACTION


In [ ]:
dfact.head()